In [1]:
import numpy as np
import pandas as pd
import os

In [ ]:
time = 2
taz = 3
indi = 5

In [7]:
np.random.binomial(2, 0.5, 5)

array([0, 0, 1, 1, 2])

In [8]:
np.random.binomial(3, 0.5, 5)

array([0, 1, 0, 2, 2])

In [9]:
a = np.array([[1,1,0,0,0,0],
             [1,0,0,1,0,0],
             [0,1,1,0,0,0],
             [0,0,1,0,0,1],
             [0,0,0,0,1,1]])

In [20]:
test = pd.DataFrame(np.vstack((np.where(a > 0)[0], np.where(a > 0)[1], a[np.where(a>0)])).T)
test.columns = ['newid', 'tsid', 'obs']

In [ ]:
def get_time_variationbyzone(data, taz, time_loc, plot_index):
    data_taz = data[data['taz']==taz]
    data_taz = data_taz.sort_values(by=['hr', 'min'])
    data_taz = data_taz.groupby(by=['hr', 'min']).sum()
    
    data_taz.reset_index(inplace=True)
    data_taz = data_taz[['hr','min','sum']]
    #data_taz
    obs_sum = np.zeros((len(time_loc)))
    hr_ = data_taz['hr'].values
    min_ = data_taz['min'].values
    sum_val = data_taz['sum'].values
    
    for i in range(len(hr_)):
        obs_sum[time_loc[str(hr_[i])+'_'+str(min_[i])]] = sum_val[i]
    #use obs/max(obs) instead of obs/sum(obs) 
    obs_per = obs_sum/(np.max(obs_sum))
    
    if plot_index == True:
        fig = plt.figure(taz)
        plt.plot(obs_sum)
    return(obs_per)

def get_sim_obs_ts(fcd_out, taz_time_per, avg_per, return_sim=True):
    taz_ = fcd_out['taz'].values
    time_5 = fcd_out['time_5'].values
    per_ts = []
    for i in range(len(taz_)):
        if taz_[i] in taz_time_per:
            per_ts.append(taz_time_per[taz_[i]][time_5[i]])
        else:
            per_ts.append(avg_per[time_5[i]])
    if return_sim == True:
        sim_obs_ts = [] 
        for i in range(len(per_ts)):
            sim_obs_ts.append(np.random.binomial(1, per_ts[i], 1)[0])
        return(sim_obs_ts)
    else:
        return(per_ts)
    
def get_simdt(fcd_out, sim_col, newuser_, loc_time):
    sim_dt = np.zeros((len(newuser_), len(loc_time)))
    ids_ = fcd_out['id'].values
    tl_id_ = fcd_out['tl_id'].values
    ct_ = fcd_out[sim_col].values
    for i in range(len(ids_)):
        sim_dt[newuser_[ids_[i]]][loc_time[tl_id_[i]]] = ct_[i]
    return(sim_dt)


#convert dt to matrix
def getfcdnorigindt(fcd_dt_path, savepath, 
                    data_obs, time_loc, taz_time_per, avg_per, 
                    save_index):
    fcd_out = pd.read_csv(fcd_dt_path, header=None)
    fcd_out.columns = ['id', 'taz', 'time']
    fcd_out.taz = fcd_out.taz.astype('int')
    fcd_out.time = fcd_out.time.astype('int')
    print('num id', len(np.unique(fcd_out.id)))

    #fcd_out.sort_values(by=['time', 'taz'])
    fcd_out['tl_id'] = fcd_out['taz'].astype(str)+'_'+((fcd_out['time']/300).astype(int)).astype(str)

    fcd_out = fcd_out[~fcd_out['time'].isin([86400, 86700])]
    time_ = np.unique((fcd_out['time']/300).astype(int))
    newuser_ = {}
    index_ = 0
    for i in np.unique(fcd_out.id):
        newuser_[i] = index_
        index_ += 1    
    ## fcd output processing
    #delete the data exceeds the 24-hr daily period
    fcd_out = fcd_out[~fcd_out['time'].isin([86400, 86700])]
    fcd_out.taz = fcd_out.taz.astype('int')
    fcd_out.time = fcd_out.time.astype('int')
    fcd_out['time_5'] = fcd_out.time/300

    fcd_out.time_5 = fcd_out.time_5.astype('int')

    obs = data_obs.groupby(by=['time']).size().values
    obs_per = obs/max(obs)

    sample_ct = lambda x: np.random.binomial(1, obs_per[x], 1)[0]

    fcd_out['sim_obs'] = fcd_out['time_5'].apply(sample_ct)

    fcd_out['tl_id'] = fcd_out['taz'].astype(str)+'_'+((fcd_out['time']/300).astype(int)).astype(str)
    
    sim_obs = get_sim_obs_ts(fcd_out, taz_time_per, avg_per, True)
    fcd_out['sim_obs_ts'] = sim_obs

    #simulated observations based on time variation
    print('sim based on time', 
          np.count_nonzero(fcd_out['sim_obs'])/fcd_out.shape[0])

    #simulated observations based on time and spatial variation
    print('sim based on time and spatial',
          np.count_nonzero(fcd_out['sim_obs_ts'])/fcd_out.shape[0])
    
    return(fcd_out)


def run_sim(csv_path, fcd_dt_path, savepath, save_index=False, plot_index=True):
    data_obs = pd.read_csv(csv_path, header=0)
    print('app based data', data_obs.head(3))
    
    newids = np.unique(data_obs.newid)
    newid_ = {}
    index_ = 0
    for i in range(len(newids)):
        newid_[newids[i]] = index_
        index_ += 1
        
    get_hr = lambda x: x.split(':')[0]
    get_min = lambda x: x.split(':')[1]

    hr_ = np.unique(data_obs['time'].astype(str).apply(get_hr).astype(int))
    min_ = np.unique(data_obs['time'].astype(str).apply(get_min).astype(int))
    taz_ = np.unique(data_obs['taz'])

    data_obs['hr'] = data_obs['time'].astype(str).apply(get_hr).astype(int)
    data_obs['min'] = data_obs['time'].astype(str).apply(get_min).astype(int)
    
    index_ = 0
    lt_loc = {}
    for taz in taz_:
        for h in hr_:
            for m in min_:
                lt_loc['4_'+str(h)+'_'+str(m)+'_'+str(taz)] = index_
                index_ += 1
    obs_mx = np.zeros((len(newid_), len(lt_loc)))

    newid_col = data_obs.newid.values
    key_col = data_obs.key.values
    ct_col = data_obs['sum'].values

    for i in range(len(newid_col)):
        obs_mx[newid_[newid_col[i]]][lt_loc[key_col[i]]] = 1

    total_ct = np.sum(obs_mx, axis=0)
    #get ct based on taz and time
    total_ct = total_ct.reshape(1, total_ct.shape[0])
    total_ct_conver = np.hsplit(total_ct, len(taz_))
    max_num = np.max(np.array(total_ct_conver), axis=2)
    
    if plot_index == True:
        plt.plot(data_obs.groupby(by=['time']).size().values)
        
    time_loc = {}
    index_ = 0
    for hr in hr_:
        for miuts in min_:
            time_loc[str(hr)+'_'+str(miuts)] = index_
            index_ += 1

    taz_time_per = {}
    avg_per = 0
    index_ = 0 
    for taz_id in taz_:
        taz_time_per[taz_id] = get_time_variationbyzone(data_obs, taz_id, time_loc, plot_index)
        if index_ == 0:
            avg_per = taz_time_per[taz_id]
        else:
            avg_per = avg_per + taz_time_per[taz_id]
        index_ += 1
        
    avg_per = avg_per/np.max(avg_per)
    if plot_index == True:
        fig = plt.figure(1)
        plt.plot(avg_per)
        
    fcd_out = getfcdnorigindt(fcd_dt_path, savepath, data_obs, time_loc, taz_time_per, avg_per, save_index=False)
    
    fcd_out['obs'] = 1
    #simulation compare
    print('sim_t vs sim_ts', mean_squared_error(fcd_out.sim_obs, fcd_out.sim_obs_ts, squared=False))
    print('obs vs sim_ts', mean_squared_error(fcd_out.obs, fcd_out.sim_obs_ts, squared=False))
    print('obs vs sim_t', mean_squared_error(fcd_out.obs, fcd_out.sim_obs, squared=False))
    
    return(fcd_out)    

In [24]:
#read app-based data
csv_path = '../data-1670309862526.csv'
fcd_dt_path = 'fcd_data_5.csv'
savepath = 'origin_dt.csv'

fcd_out = run_sim(csv_path, fcd_dt_path, savepath, save_index=False, plot_index=False)
fcd_out.head(3)

In [26]:
path = "G:/My Drive/2021/Bias/sumo_simulation/appsim"
os.chdir(path)

In [30]:
def revise_fcd_dt(fcd_outpath):
    fcd_outdt = pd.read_csv(fcd_outpath, header=None)
    fcd_outdt.columns = ['id', 'TAZ', 'time']
    fcd_outdt['time'] = fcd_outdt['time'].astype('int')
    fcd_outdt['hr'] = (fcd_outdt['time'].values/3600).astype(int)
    fcd_outdt['minus'] = (fcd_outdt['time'].values- fcd_outdt['hr'].values*3600)/60
    fcd_outdt['time_30'] = (fcd_outdt['time']/30).astype(int)
    fcd_outdt['time_5'] = (fcd_outdt['time']/300).astype(int)
    return(fcd_outdt)

def revise_userid(fcd_out):
    newuser_ = {}
    index_ = 0
    for i in np.unique(fcd_out.id):
        newuser_[i] = index_
        index_ += 1
    
    get_newid = lambda x: newuser_[x]
    fcd_out['newid'] = fcd_out['id'].apply(get_newid)
    return(fcd_out)

fcd_out_30 = revise_fcd_dt('fcd_data_5_30sec.csv')
fcd_out_30 = fcd_out_30[fcd_out_30['time_5'] < 288]
fcd_out_30 = fcd_out_30.sort_values(by = ['id', 'time'])

fcd_out_30 = revise_userid(fcd_out_30)
index_ = 0
taz_index_dict = {}
for i in np.unique(fcd_out_30.TAZ):
    taz_index_dict[i] = index_
    index_+= 1
    
index_taz_dict = {}
for i in taz_index_dict:
    index_taz_dict[taz_index_dict[i]] = i
    
get_taz_index = lambda x: taz_index_dict[x]
fcd_out_30['taz_index'] = fcd_out_30['TAZ'].apply(get_taz_index)

In [80]:
len(np.unique(fcd_out_30['TAZ']))

168

In [95]:
def get_dataobs(csv_path, fcd_out_dt):
    data_obs = pd.read_csv(csv_path, header=0)
    hr_ = np.unique(data_obs['time'].astype(str).apply(get_hr).astype(int))
    min_ = np.unique(data_obs['time'].astype(str).apply(get_min).astype(int))
    #taz_ = np.unique(data_obs['taz'])

    data_obs['hr'] = data_obs['time'].astype(str).apply(get_hr).astype(int)
    data_obs['min'] = data_obs['time'].astype(str).apply(get_min).astype(int)
    data_obs = data_obs[data_obs['taz'].isin(np.unique(fcd_out_30.TAZ))]
    data_obs['taz_index'] = data_obs['taz'].apply(get_taz_index)
    data_obs['time_5'] = (data_obs['hr']*12+data_obs['min']/5).astype(int)
    return(data_obs)

In [220]:
data_obs = get_dataobs(csv_path, fcd_out_30)
time_per = data_obs.groupby(by = ['time_5']).size().values
time_per = time_per/np.max(time_per)
avg_time_per = np.mean(time_per)

#get per based on time and spatial info
st_obs = data_obs.groupby(by=['taz_index', 'time_5']).size().to_frame()
st_obs.reset_index(inplace=True)
st_obs.columns = ['taz_index', 'time_5', 'obs']

taz_val = st_obs['taz_index'].values
time_val = st_obs['time_5'].values
obs = st_obs['obs'].values

st_per = np.zeros((len(np.unique(taz_val)), len(np.unique(time_val))))

re_revise_taz = {}
index_ = 0
for i in np.unique(taz_val):
    re_revise_taz[i] = index_
    index_ += 1
    
get_re_index = lambda x: re_revise_taz[x]
st_obs['taz_re_index'] = st_obs['taz_index'].apply(get_re_index)
taz_val = st_obs['taz_re_index'].values

st_per[taz_val, time_val] = obs

st_per = np.divide(st_per, np.tile(np.max(st_per, axis=1).reshape(len(np.unique(taz_val)),1), 288))
st_per_avg = np.mean(st_per, axis=0)

get_re_index = lambda x: re_revise_taz[x] if x in re_revise_taz else -1
fcd_out_30['taz_re_index'] = fcd_out_30['taz_index'].apply(get_re_index)

In [212]:
fcd_obs = fcd_out_30.groupby(by = ['newid', 'taz_re_index', 'time_5']).size().to_frame()
fcd_obs.reset_index(inplace=True)

fcd_obs.columns = ['newid', 'taz_re_index', 'time_5', 'obs']

In [225]:
#conduct_sim
time_5 = fcd_obs['time_5'].values
obs = fcd_obs['obs'].values
taz_ = fcd_obs['taz_re_index'].values

sim_obs_t = []
sim_obs_ts = []
for i in range(len(time_5)):
    sim_obs_t.append(np.sum(np.random.binomial(1, time_per[time_5[i]], size=obs[i])))
    if taz_[i] != -1:
        sim_obs_ts.append(np.sum(np.random.binomial(1, st_per[taz_[i],time_5[i]], size=obs[i])))
    else:
        sim_obs_ts.append(np.sum(np.random.binomial(1, st_per_avg[time_5[i]], size=obs[i])))
        
        

In [227]:
168*288

48384

In [240]:
index_ = 300
for i in range(168*288):
    if i < index_:
        print(i%288, i//288)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 0
177 0
178 0
179 0
180 0
181 0
182 0
183 0
184 0


In [241]:
a = [1,2,3]
np.array(a)

array([1, 2, 3])

In [217]:
st_per[0, 1]

0.09090909090909091

In [159]:
obs[0]

9

In [158]:
fcd_obs.head(3)

,newid,taz_re_index,time_5,obs
0,0,1,6,9
1,0,1,7,3
2,0,108,12,4


In [156]:
sim_obs_t

[2,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 5,
 1,
 2,
 2,
 0,
 1,
 9,
 2,
 6,
 5,
 1,
 3,
 4,
 5,
 2,
 3,
 7,
 1,
 2,
 1,
 1,
 2,
 4,
 2,
 1,
 7,
 1,
 2,
 7,
 1,
 1,
 6,
 1,
 2,
 4,
 1,
 9,
 2,
 2,
 4,
 2,
 3,
 5,
 1,
 1,
 1,
 6,
 5,
 1,
 1,
 3,
 1,
 1,
 8,
 1,
 4,
 3,
 6,
 2,
 4,
 4,
 2,
 3,
 3,
 2,
 1,
 5,
 2,
 2,
 3,
 2,
 5,
 1,
 1,
 1,
 4,
 3,
 6,
 1,
 4,
 4,
 3,
 4,
 2,
 1,
 4,
 2,
 1,
 3,
 5,
 1,
 1,
 4,
 2,
 2,
 3,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 2,
 2,
 1,
 6,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 3,
 1,
 2,
 6,
 2,
 4,
 4,
 3,
 4,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 2,
 3,
 4,
 3,
 1,
 1,
 3,
 4,
 2,
 1,
 7,
 5,
 1,
 1,
 1,
 2,
 6,
 1,
 1,
 4,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 3,
 4,
 1,
 1,
 5,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 4,
 2,
 2,
 1,
 2,
 2,
 6,
 0,
 1,


In [152]:
time_per

array([0.20337739, 0.18135095, 0.19016153, 0.18502203, 0.19456681,
       0.16666667, 0.1681351 , 0.14757709, 0.15785609, 0.16299559,
       0.16299559, 0.1681351 , 0.17621145, 0.1732746 , 0.15051395,
       0.15271659, 0.15785609, 0.14464023, 0.14537445, 0.12041116,
       0.14023495, 0.13069016, 0.14977974, 0.13656388, 0.14023495,
       0.13582966, 0.14610866, 0.13582966, 0.12995595, 0.12555066,
       0.14390602, 0.12922173, 0.13582966, 0.14390602, 0.13362702,
       0.13142438, 0.12481645, 0.12408223, 0.11600587, 0.12922173,
       0.13656388, 0.13142438, 0.14170338, 0.1174743 , 0.1071953 ,
       0.11380323, 0.12848752, 0.1123348 , 0.14757709, 0.12041116,
       0.12041116, 0.12408223, 0.1328928 , 0.13069016, 0.12922173,
       0.14023495, 0.15345081, 0.17033774, 0.18061674, 0.16519824,
       0.16446402, 0.15124816, 0.14537445, 0.18502203, 0.18722467,
       0.19970631, 0.21071953, 0.21145374, 0.23788546, 0.26358297,
       0.25844347, 0.25403818, 0.27606461, 0.2753304 , 0.33186

(162, 288)